# Scrapy 프레임워크 

> - Scrapy 개요
- Spider 만들고 실행하기
- 실전적인 크롤링
- 추출한 데이터 처리하기
- Scrapy 설정
- Scrapy 확장하기
- 크롤링으로 데이터 수집하고 활용하기
- 이미지 수집과 활용

## Scrapy 개요 
- 파이썬에서 제공하는 크롤링/스크레이핑 프레임워크
- Scrapy는 파이썬2에만 지원되었지만, 2016년 5월 배포된 1.1버전부터 파이썬3에도 지원 시작

In [ ]:
! pip install scrapy

In [ ]:
! scrapy version


**c6-01_myspider.py** 
> Scrapinghub 블로그에서 글의 타이틀을 추출하는 Spider

> - 아래 myspider.py 코드는 Scrapy 사이트에서 제공하는 공식 예제 코드에 주석을 붙인 것이다.
- 실행 : scrapy runspider myspider.py -o items.j1

In [ ]:
%%writefile  ./modules/myspider.py
import scrapy

class BlogSpider(scrapy.Spider):
    # spider의 이름
    name = 'blogspider'

    # 크롤링을 시작할 URL 리스트
    start_urls = ['https://blog.scrapinghub.com']

    def parse(self, response):
        """
        최상위 페이지에서 카테고리 페이지의 링크를 추출합니다.
        """
        for url in response.css('ul li a::attr("href")').re('.*/category/.*'):
            yield scrapy.Request(response.urljoin(url), self.parse_titles)

    def parse_titles(self, response):
        """
        카페고리 페이지에서 카테고리 타이틀을 모두 추출합니다.
        """
        for post_title in response.css('div.entries > ul > li a::text').extract():
            yield {'title': post_title}
            
            

In [ ]:
! scrapy runspider ./modules/myspider.py -o ./modules/data/items.jl

In [ ]:
! type .\modules\data\items.jl

### yield generator test source

In [ ]:
#python 3 version source
#yield generator test source
#yield_Basic_Test.py
 
def number_generator(n):
    print("Function Start")
    while n < 6:
        yield  {"n" : n}
        n += 1
    print("Function End")
     
if __name__ == "__main__":
    for i in number_generator(0):
        print(i)
         

## Spider 만들고 실행하기 
- 타겟사이트 : https://www.engadget.com
- 명령어실행 : (base) C:\ > scrapy startproject myproject
- Item만들기

In [ ]:
# scrapy.cfg

# 페이지 다운로드 간격을 1초로 지정
DOWNLOAD_DELAY = 1

In [ ]:
# myproject/items.py
import scrapy

class Headline(scrapy.Item):
    """
    뉴스 헤드라인을 나타내는 Item 객체
    """
    title = scrapy.Field()
    body = scrapy.Field()

In [ ]:
# myproject/spiders/news.py
import scrapy

class NewsSpider(scrapy.Spider):
    name = 'news'
    allowed_domains = ['engadget.com']
    start_urls = ['http://engadget.com/']

    def parse(self, response):
        """
        메인 페이지의 토픽 목록에서 링크를 추출하고 출력합니다.
        """
        link = response.css('a.o-hit__link::attr("href")').extract()
        link = filter(lambda x : x != "#", link)
        link = list(link)
        print(link)

In [ ]:
# myproject/spiders/news.py
import scrapy


class NewsSpider(scrapy.Spider):
    name = 'news'
    allowed_domains = ['engadget.com']
    start_urls = ['http://engadget.com/']

    def parse(self, response):
        """
        메인 페이지의 토픽 목록에서 링크를 추출하고 출력합니다.
        """
        link = response.css('a.o-hit__link::attr("href")').extract()
        # link = filter(lambda x : x != "#", link)
        # link = list(link)
        # print(link)

        for url in link:
            # 광고 페이지 제외
            if url.find("products") == 1:
                continue
            # 의미 없는 페이지 제외
            if url == "#":
                continue
            # 기사 페이지
            yield scrapy.Request(response.urljoin(url), self.parse_topics)

    def parse_topics(self, response):
        pass

In [ ]:
# -*- coding: utf-8 -*-
import scrapy
from webcrawler.wc06_2.myproject.items import Headline

class NewsSpider(scrapy.Spider):
    name = 'news'
    allowed_domains = ['engadget.com']
    start_urls = ['http://engadget.com/']

    def parse(self, response):
        """
        메인 페이지의 토픽 목록에서 링크를 추출하고 출력합니다.
        """
        link = response.css('a.o-hit__link::attr("href")').extract()
        # link = filter(lambda x : x != "#", link)
        # link = list(link)
        # print(link)

        for url in link:
            # 광고 페이지 제외
            if url.find("products") == 1:
                continue
            # 의미 없는 페이지 제외
            if url == "#":
                continue
            # 기사 페이지
            yield scrapy.Request(response.urljoin(url), self.parse_topics)

    def parse_topics(self, response):
        pass
        item = Headline()
        item['title'] = response.css('head title::text').extract_first()
        item['body'] = " ".join(response.css('.o-article_block p')\
            .xpath('string()')\
            .extract())
        yield item

C:\webcralwer\myproject> scrapy shell https://www.engadget.com/2017/08/17/hyundai-shifts-focus-from-fuel-cell-cars-to-evs/